# Jour 3-4 : CI/CD avec GitHub Actions

## Objectifs des jours 3-4

Automatiser complètement vos vérifications de qualité avec GitHub Actions, créer des workflows qui testent votre code à chaque modification, implémenter le déploiement continu, et établir un processus de collaboration d'équipe robuste.

## 1. Introduction aux workflows automatisés

### Comprendre GitHub Actions par l'analogie de l'usine

GitHub Actions fonctionne comme une usine automatisée ultra-moderne. Chaque fois que vous livrez des matières premières (votre code), l'usine démarre automatiquement une chaîne de production. Elle vérifie la qualité des matières premières, assemble le produit, teste le résultat, et si tout va bien, livre le produit fini. Si quelque chose ne va pas à n'importe quelle étape, la production s'arrête et vous êtes alerté.

**Avantages de l'automatisation complète :**

L'automatisation élimine les erreurs humaines et garantit la cohérence. Plus besoin de se souvenir de lancer les tests avant de pousser le code. Plus de risque d'oublier une vérification importante. GitHub Actions devient votre assistant personnel qui ne dort jamais et ne fait jamais d'erreur.

### Création de la structure GitHub Actions

Commençons par créer la structure nécessaire pour vos workflows.

In [ ]:
# Création de la structure GitHub Actions
mkdir -p .github/workflows
touch .github/workflows/quality-check.yml

### Votre premier workflow GitHub Actions

Créons ensemble votre premier workflow qui automatise toutes les vérifications que vous faisiez manuellement les jours précédents.

In [ ]:
# Contenu du fichier .github/workflows/quality-check.yml
workflow_content = '''
# .github/workflows/quality-check.yml
name: Contrôle Qualité

# Déclencheurs du workflow
on:
  # À chaque push sur main ou develop
  push:
    branches: [ main, develop ]
  # À chaque pull request vers main
  pull_request:
    branches: [ main ]

# Variables d'environnement globales
env:
  PYTHON_VERSION: '3.11'

# Les tâches à exécuter
jobs:
  quality-check:
    name: Vérification de la qualité du code
    runs-on: ubuntu-latest
    
    steps:
    # Étape 1: Récupérer le code source
    - name: Récupération du code
      uses: actions/checkout@v4
    
    # Étape 2: Configurer Python
    - name: Configuration Python ${{ env.PYTHON_VERSION }}
      uses: actions/setup-python@v4
      with:
        python-version: ${{ env.PYTHON_VERSION }}
    
    # Étape 3: Installer Poetry
    - name: Installation Poetry
      uses: snok/install-poetry@v1
      with:
        version: latest
        virtualenvs-create: true
        virtualenvs-in-project: true
    
    # Étape 4: Cache pour accélérer les builds
    - name: Cache des dépendances Poetry
      uses: actions/cache@v3
      with:
        path: .venv
        key: venv-${{ runner.os }}-${{ hashFiles('**/poetry.lock') }}
        restore-keys: |
          venv-${{ runner.os }}-
    
    # Étape 5: Installer les dépendances
    - name: Installation des dépendances
      run: poetry install --no-interaction --no-ansi
    
    # Étape 6: Vérification avec Ruff
    - name: Vérification du style avec Ruff
      run: |
        poetry run ruff check src/ tests/
        poetry run ruff format --check src/ tests/
    
    # Étape 7: Exécution des tests
    - name: Exécution des tests unitaires
      run: |
        poetry run pytest tests/ \
          --cov=src \
          --cov-report=xml \
          --cov-report=term-missing \
          --junitxml=pytest-results.xml
    
    # Étape 8: Upload des résultats de couverture
    - name: Upload couverture vers Codecov
      uses: codecov/codecov-action@v3
      if: always()
      with:
        file: ./coverage.xml
        fail_ci_if_error: false
    
    # Étape 9: Publication des résultats de tests
    - name: Publication des résultats de tests
      uses: dorny/test-reporter@v1
      if: always()
      with:
        name: Résultats des tests pytest
        path: pytest-results.xml
        reporter: java-junit
'''

# Écriture du fichier
with open('.github/workflows/quality-check.yml', 'w') as f:
    f.write(workflow_content)
    
print("Workflow de base créé avec succès !")

### Explication détaillée du workflow

**Section `on` - Les déclencheurs :** Ce workflow s'exécute automatiquement dans deux situations : quand vous poussez du code sur les branches principales (main, develop) et quand quelqu'un crée une pull request vers main. Cette configuration garantit que tout code qui rejoint le projet principal a été vérifié.

**Section `env` - Variables globales :** Définir la version Python comme variable globale facilite la maintenance. Si vous voulez changer de version Python, vous n'avez qu'un seul endroit à modifier.

**Étapes du job expliquées :**

- L'étape `actions/checkout@v4` télécharge votre code source dans la machine virtuelle GitHub Actions
- L'étape `actions/setup-python@v4` installe la version exacte de Python
- L'étape `snok/install-poetry@v1` installe Poetry avec la configuration optimale
- L'étape de cache `actions/cache@v3` sauvegarde vos dépendances entre les exécutions

### Workflow avancé avec matrice de tests

Une fois votre workflow de base fonctionnel, enrichissons-le pour tester sur plusieurs environnements simultanément.

In [ ]:
# Workflow avec matrice de tests
matrix_workflow = '''
# .github/workflows/comprehensive-test.yml
name: Tests Complets Multi-Environnements

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  test-matrix:
    name: Tests Python ${{ matrix.python-version }} sur ${{ matrix.os }}
    runs-on: ${{ matrix.os }}
    
    strategy:
      # Continue même si un environnement échoue
      fail-fast: false
      matrix:
        os: [ubuntu-latest, windows-latest, macos-latest]
        python-version: ['3.11', '3.12']
        # Exclusions pour optimiser les ressources
        exclude:
          - os: macos-latest
            python-version: '3.12'  # Économise des minutes CI
    
    steps:
    - uses: actions/checkout@v4
    
    - name: Configuration Python ${{ matrix.python-version }}
      uses: actions/setup-python@v4
      with:
        python-version: ${{ matrix.python-version }}
    
    - name: Installation Poetry
      uses: snok/install-poetry@v1
    
    - name: Cache dépendances
      uses: actions/cache@v3
      with:
        path: .venv
        key: venv-${{ matrix.os }}-${{ matrix.python-version }}-${{ hashFiles('**/poetry.lock') }}
    
    - name: Installation dépendances
      run: poetry install --no-interaction
    
    - name: Tests avec pytest
      run: |
        poetry run pytest tests/ \
          --cov=src \
          --cov-report=xml \
          -v
    
    - name: Vérification Ruff
      run: |
        poetry run ruff check src/ tests/
        poetry run ruff format --check src/ tests/
    
    # Upload de couverture seulement pour Ubuntu + Python 3.11
    - name: Upload couverture
      if: matrix.os == 'ubuntu-latest' && matrix.python-version == '3.11'
      uses: codecov/codecov-action@v3
      with:
        file: ./coverage.xml
'''

with open('.github/workflows/comprehensive-test.yml', 'w') as f:
    f.write(matrix_workflow)
    
print("Workflow avec matrice de tests créé !")

**Pourquoi utiliser une matrice de tests :**

La matrice révèle des problèmes de compatibilité invisibles en développement local. Par exemple, un chemin de fichier utilisant `/` fonctionne sur Linux et macOS mais peut échouer sur Windows. Une fonction qui marche avec Python 3.11 pourrait utiliser une syntaxe incompatible avec Python 3.12.

## 2. Pipeline de déploiement continu

### Architecture d'un pipeline complet

Un pipeline de déploiement continu suit une progression logique : vérification de la qualité, tests unitaires, tests d'intégration, construction de l'artefact de déploiement, et enfin déploiement si tout va bien.

In [ ]:
# Pipeline de déploiement continu complet
cd_pipeline = '''
# .github/workflows/cd-pipeline.yml
name: Pipeline de Déploiement Continu

on:
  push:
    branches: [ main ]
    tags: [ 'v*' ]

env:
  PYTHON_VERSION: '3.11'
  REGISTRY: ghcr.io
  IMAGE_NAME: ${{ github.repository }}

jobs:
  # Job 1: Vérifications rapides
  quick-checks:
    name: Vérifications Rapides
    runs-on: ubuntu-latest
    
    steps:
    - uses: actions/checkout@v4
    
    - name: Configuration Python
      uses: actions/setup-python@v4
      with:
        python-version: ${{ env.PYTHON_VERSION }}
    
    - name: Installation Poetry
      uses: snok/install-poetry@v1
    
    - name: Cache dépendances
      uses: actions/cache@v3
      with:
        path: .venv
        key: venv-${{ runner.os }}-${{ hashFiles('**/poetry.lock') }}
    
    - name: Installation dépendances
      run: poetry install --no-interaction
    
    - name: Vérification syntaxe Python
      run: python -m py_compile src/**/*.py
    
    - name: Vérification imports
      run: poetry run python -c "import src"
    
    - name: Linting avec Ruff
      run: poetry run ruff check src/ tests/
    
    - name: Formatage avec Ruff
      run: poetry run ruff format --check src/ tests/

  # Job 2: Tests complets
  comprehensive-tests:
    name: Tests Complets
    runs-on: ubuntu-latest
    needs: quick-checks  # Attend que quick-checks réussisse
    
    steps:
    - uses: actions/checkout@v4
    
    - name: Configuration Python
      uses: actions/setup-python@v4
      with:
        python-version: ${{ env.PYTHON_VERSION }}
    
    - name: Installation Poetry
      uses: snok/install-poetry@v1
    
    - name: Cache dépendances
      uses: actions/cache@v3
      with:
        path: .venv
        key: venv-${{ runner.os }}-${{ hashFiles('**/poetry.lock') }}
    
    - name: Installation dépendances
      run: poetry install --no-interaction
    
    - name: Tests unitaires
      run: |
        poetry run pytest tests/unit/ \
          --cov=src \
          --cov-report=xml \
          --cov-fail-under=80 \
          -v
    
    - name: Tests d'intégration
      run: |
        poetry run pytest tests/integration/ \
          -v \
          --tb=short
    
    - name: Tests de performance
      run: |
        poetry run pytest tests/ \
          -m "slow" \
          --benchmark-only \
          --benchmark-json=benchmark.json
    
    - name: Upload résultats
      uses: actions/upload-artifact@v3
      with:
        name: test-results
        path: |
          coverage.xml
          benchmark.json

  # Job 3: Analyse de sécurité
  security-analysis:
    name: Analyse de Sécurité
    runs-on: ubuntu-latest
    needs: quick-checks
    
    steps:
    - uses: actions/checkout@v4
    
    - name: Configuration Python
      uses: actions/setup-python@v4
      with:
        python-version: ${{ env.PYTHON_VERSION }}
    
    - name: Installation Poetry
      uses: snok/install-poetry@v1
    
    - name: Installation dépendances
      run: poetry install --no-interaction
    
    - name: Audit de sécurité avec Safety
      run: |
        poetry add --group dev safety
        poetry run safety check --json --output safety-report.json
      continue-on-error: true
    
    - name: Scan des secrets
      uses: trufflesecurity/trufflehog@main
      with:
        path: ./
        base: main
        head: HEAD
        extra_args: --debug --only-verified

  # Job 4: Construction et publication
  build-and-deploy:
    name: Construction et Déploiement
    runs-on: ubuntu-latest
    needs: [comprehensive-tests, security-analysis]
    if: github.ref == 'refs/heads/main' || startsWith(github.ref, 'refs/tags/')
    
    steps:
    - uses: actions/checkout@v4
    
    - name: Configuration Python
      uses: actions/setup-python@v4
      with:
        python-version: ${{ env.PYTHON_VERSION }}
    
    - name: Installation Poetry
      uses: snok/install-poetry@v1
    
    - name: Construction du package
      run: |
        poetry build
        ls -la dist/
    
    - name: Publication sur PyPI (si tag)
      if: startsWith(github.ref, 'refs/tags/')
      env:
        POETRY_PYPI_TOKEN_PYPI: ${{ secrets.PYPI_TOKEN }}
      run: |
        poetry publish
    
    - name: Création de la release GitHub
      if: startsWith(github.ref, 'refs/tags/')
      uses: softprops/action-gh-release@v1
      with:
        files: dist/*
        generate_release_notes: true
'''

with open('.github/workflows/cd-pipeline.yml', 'w') as f:
    f.write(cd_pipeline)
    
print("Pipeline de déploiement continu créé !")

### Architecture du pipeline expliquée

Ce pipeline utilise une approche en jobs parallèles après les vérifications rapides. Cette architecture optimise le temps d'exécution :

- **Le job `quick-checks`** : Exécute les vérifications les plus rapides en premier (< 30 secondes)
- **Le job `comprehensive-tests`** : Tests complets avec mesure de couverture et benchmarks
- **Le job `security-analysis`** : Vérifications de sécurité avec Safety et TruffleHog
- **Le job `build-and-deploy`** : Construction et publication (seulement si tous les tests passent)

### Gestion des environnements et secrets

GitHub permet de créer des environnements (development, staging, production) avec des règles de protection et des secrets spécifiques.

In [ ]:
# Exemple d'utilisation d'environnements
environments_example = '''
jobs:
  deploy-staging:
    name: Déploiement Staging
    runs-on: ubuntu-latest
    environment: staging
    needs: comprehensive-tests
    
    steps:
    - name: Déploiement vers staging
      env:
        STAGING_API_KEY: ${{ secrets.STAGING_API_KEY }}
        STAGING_DATABASE_URL: ${{ secrets.STAGING_DATABASE_URL }}
      run: |
        echo "Déploiement vers l'environnement de staging"
        # Scripts de déploiement ici

  deploy-production:
    name: Déploiement Production
    runs-on: ubuntu-latest
    environment: production
    needs: deploy-staging
    if: github.ref == 'refs/heads/main'
    
    steps:
    - name: Déploiement vers production
      env:
        PROD_API_KEY: ${{ secrets.PROD_API_KEY }}
        PROD_DATABASE_URL: ${{ secrets.PROD_DATABASE_URL }}
      run: |
        echo "Déploiement vers la production"
        # Scripts de déploiement production
'''

print("Exemple de gestion d'environnements :")
print(environments_example)

### Bonnes pratiques pour les secrets

Les secrets GitHub sont chiffrés et ne sont jamais exposés dans les logs. Configurez-les dans Settings → Secrets and variables → Actions de votre repository.

**Bonnes pratiques :**
- Utilisez des noms explicites comme `PROD_DATABASE_URL` plutôt que `DB_URL`
- Séparez les secrets par environnement
- Utilisez des tokens avec permissions minimales
- Rotez régulièrement vos secrets

## 3. Monitoring et notifications

### Système de notifications intelligent

Configurons un système de notifications qui vous informe des résultats sans vous spammer.

In [ ]:
# Système de notifications
notifications_workflow = '''
# Job de notification à la fin du pipeline
  notify-results:
    name: Notifications
    runs-on: ubuntu-latest
    needs: [comprehensive-tests, security-analysis, build-and-deploy]
    if: always()  # S'exécute même si d'autres jobs échouent
    
    steps:
    - name: Détermination du statut global
      id: status
      run: |
        if [[ "${{ needs.comprehensive-tests.result }}" == "success" && 
              "${{ needs.security-analysis.result }}" == "success" && 
              ("${{ needs.build-and-deploy.result }}" == "success" || "${{ needs.build-and-deploy.result }}" == "skipped") ]]; then
          echo "status=success" >> $GITHUB_OUTPUT
          echo "message=✅ Pipeline réussi ! Tous les tests passent." >> $GITHUB_OUTPUT
        else
          echo "status=failure" >> $GITHUB_OUTPUT
          echo "message=❌ Pipeline échoué. Vérifiez les logs pour plus de détails." >> $GITHUB_OUTPUT
        fi
    
    - name: Notification Slack (succès)
      if: steps.status.outputs.status == 'success'
      uses: 8398a7/action-slack@v3
      with:
        status: success
        channel: '#data-engineering'
        username: 'GitHub Actions'
        icon_emoji: ':rocket:'
        title: 'Déploiement réussi'
        message: |
          🎉 Le pipeline ${{ github.repository }} a réussi !
          
          📊 **Détails :**
          • Branche : ${{ github.ref_name }}
          • Commit : ${{ github.sha }}
          • Auteur : ${{ github.actor }}
          
          🔗 [Voir les détails](${{ github.server_url }}/${{ github.repository }}/actions/runs/${{ github.run_id }})
      env:
        SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK }}
    
    - name: Notification Slack (échec)
      if: steps.status.outputs.status == 'failure'
      uses: 8398a7/action-slack@v3
      with:
        status: failure
        channel: '#data-engineering'
        username: 'GitHub Actions'
        icon_emoji: ':warning:'
        title: 'Pipeline échoué'
        message: |
          🚨 Le pipeline ${{ github.repository }} a échoué !
          
          📊 **Détails :**
          • Branche : ${{ github.ref_name }}
          • Commit : ${{ github.sha }}
          • Auteur : ${{ github.actor }}
          
          🔧 **Actions requises :**
          • Vérifiez les logs d'erreur
          • Corrigez les problèmes identifiés
          • Relancez le pipeline
          
          🔗 [Voir les logs d'erreur](${{ github.server_url }}/${{ github.repository }}/actions/runs/${{ github.run_id }})
      env:
        SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK }}
    
    - name: Notification email (échecs critiques)
      if: steps.status.outputs.status == 'failure' && github.ref == 'refs/heads/main'
      uses: dawidd6/action-send-mail@v3
      with:
        server_address: smtp.gmail.com
        server_port: 587
        username: ${{ secrets.EMAIL_USERNAME }}
        password: ${{ secrets.EMAIL_PASSWORD }}
        subject: '🚨 URGENT: Pipeline production échoué'
        to: 'team-data@company.com'
        from: 'github-actions@company.com'
        body: |
          Le pipeline de production a échoué sur la branche main.
          
          Repository: ${{ github.repository }}
          Commit: ${{ github.sha }}
          Auteur: ${{ github.actor }}
          
          Action immédiate requise.
          
          Logs: ${{ github.server_url }}/${{ github.repository }}/actions/runs/${{ github.run_id }}
'''

print("Système de notifications configuré !")
print(notifications_workflow)

### Métriques et tableaux de bord

Créons un système de collecte de métriques pour suivre la santé de votre pipeline.

In [ ]:
# Job de collecte de métriques
metrics_job = '''
  collect-metrics:
    name: Collecte de Métriques
    runs-on: ubuntu-latest
    needs: [comprehensive-tests]
    if: always()
    
    steps:
    - uses: actions/checkout@v4
    
    - name: Calcul des métriques
      id: metrics
      run: |
        # Durée du pipeline
        start_time="${{ github.event.head_commit.timestamp }}"
        current_time=$(date -u +"%Y-%m-%dT%H:%M:%SZ")
        
        # Statut des tests
        test_status="${{ needs.comprehensive-tests.result }}"
        test_success=$([[ "$test_status" == "success" ]] && echo "1" || echo "0")
        
        # Informations sur le commit
        commit_sha="${{ github.sha }}"
        branch="${{ github.ref_name }}"
        author="${{ github.actor }}"
        
        # Sauvegarde des métriques
        cat > metrics.json << EOF
        {
          "timestamp": "$current_time",
          "repository": "${{ github.repository }}",
          "branch": "$branch",
          "commit_sha": "$commit_sha",
          "author": "$author",
          "test_success": $test_success,
          "pipeline_duration_minutes": 10,
          "coverage_percentage": 85
        }
        EOF
        
        echo "metrics_file=metrics.json" >> $GITHUB_OUTPUT
    
    - name: Envoi des métriques vers InfluxDB
      env:
        INFLUXDB_URL: ${{ secrets.INFLUXDB_URL }}
        INFLUXDB_TOKEN: ${{ secrets.INFLUXDB_TOKEN }}
      run: |
        # Envoi des métriques vers votre système de monitoring
        curl -X POST "$INFLUXDB_URL/api/v2/write" \
          -H "Authorization: Token $INFLUXDB_TOKEN" \
          -H "Content-Type: text/plain" \
          --data-binary @metrics.json
'''

print("Job de collecte de métriques :")
print(metrics_job)

## Exercices pratiques

### Exercice 1: Créer votre premier workflow

Créez un workflow GitHub Actions qui :
1. Se déclenche sur les push vers main
2. Installe Python et Poetry
3. Exécute Ruff et pytest
4. Publie les résultats de couverture

In [ ]:
# Exercice 1: Votre premier workflow
# TODO: Créez le fichier .github/workflows/mon-premier-workflow.yml
# Utilisez les exemples ci-dessus comme base

# Vérifiez que la structure est correcte
import os
if os.path.exists('.github/workflows'):
    print("✅ Dossier .github/workflows existe")
    workflows = os.listdir('.github/workflows')
    print(f"📁 Workflows trouvés: {workflows}")
else:
    print("❌ Dossier .github/workflows n'existe pas")

### Exercice 2: Pipeline avec matrice de tests

Modifiez votre workflow pour tester sur plusieurs versions de Python et systèmes d'exploitation.

In [ ]:
# Exercice 2: Matrice de tests
# TODO: Ajoutez une matrice de tests à votre workflow
# Testez sur Python 3.11 et 3.12
# Testez sur Ubuntu et Windows

matrix_config = {
    "os": ["ubuntu-latest", "windows-latest"],
    "python-version": ["3.11", "3.12"]
}

print("Configuration de matrice suggérée:")
print(f"OS: {matrix_config['os']}")
print(f"Python: {matrix_config['python-version']}")

### Exercice 3: Notifications personnalisées

Configurez des notifications Slack ou email pour votre pipeline.

In [ ]:
# Exercice 3: Notifications
# TODO: Ajoutez un job de notification à votre pipeline
# Configurez les secrets nécessaires dans GitHub

secrets_needed = [
    "SLACK_WEBHOOK_URL",
    "EMAIL_USERNAME", 
    "EMAIL_PASSWORD"
]

print("Secrets à configurer dans GitHub:")
for secret in secrets_needed:
    print(f"- {secret}")

print("\nPour configurer les secrets:")
print("1. Allez dans Settings → Secrets and variables → Actions")
print("2. Cliquez sur 'New repository secret'")
print("3. Ajoutez chaque secret avec sa valeur")

## Résumé des concepts clés

### Points essentiels à retenir

1. **GitHub Actions = Usine automatisée** qui vérifie, teste et déploie votre code
2. **Workflows en jobs parallèles** pour optimiser le temps d'exécution
3. **Matrice de tests** pour détecter les problèmes de compatibilité
4. **Gestion des secrets** pour sécuriser vos déploiements
5. **Notifications intelligentes** pour rester informé sans spam
6. **Métriques de pipeline** pour suivre la santé de votre CI/CD

### Prochaines étapes

1. Implémentez votre premier workflow
2. Ajoutez progressivement la complexité (matrice, sécurité, notifications)
3. Mesurez et optimisez les performances de votre pipeline
4. Établissez un processus de collaboration d'équipe robuste

In [ ]:
# Vérification finale de votre setup
import os
import yaml

def check_workflow_setup():
    """Vérifie que votre setup GitHub Actions est correct"""
    
    checks = {
        "Dossier .github/workflows existe": os.path.exists('.github/workflows'),
        "Au moins un workflow YAML": False,
        "Workflow contient des jobs": False
    }
    
    if checks["Dossier .github/workflows existe"]:
        workflows = [f for f in os.listdir('.github/workflows') if f.endswith('.yml')]
        checks["Au moins un workflow YAML"] = len(workflows) > 0
        
        if workflows:
            # Vérifier le contenu du premier workflow
            try:
                with open(f'.github/workflows/{workflows[0]}', 'r') as f:
                    content = f.read()
                    checks["Workflow contient des jobs"] = 'jobs:' in content
            except:
                pass
    
    print("🔍 Vérification de votre setup GitHub Actions:")
    for check, passed in checks.items():
        status = "✅" if passed else "❌"
        print(f"{status} {check}")
    
    if all(checks.values()):
        print("\n🎉 Félicitations ! Votre setup GitHub Actions est prêt !")
    else:
        print("\n⚠️ Quelques éléments manquent. Référez-vous aux exercices ci-dessus.")

check_workflow_setup()